In [1]:
#Import preprocessed data
#Import packages
import tensorflow as tf
from tensorflow import keras
from scipy.io import loadmat, savemat
from os import listdir
from os.path import dirname, join as join
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

print(tf.__version__)


#import visual representation - mnist
mnist =  keras.datasets.mnist
(train_imgs_mnist, train_labs_mnist), (test_imgs_mnist, test_labs_mnist) = mnist.load_data()

train_imgs_mnist = train_imgs_mnist/255.0
test_imgs_mnist = test_imgs_mnist/255.0

#import cochleagrams 
def load_files(dir): #coch=15,53,
    filenames = listdir(dir)
    files = np.zeros((len(filenames), 15, 53))
    labels = []
    for i in range(len(filenames)):
        file = np.load(join(dir, filenames[i]))
        files[i] = file
        label = int(filenames[i][0]) #to get the 6th character in the string, which gives the digit pronounced
        labels.append(label)
    labels = np.array(labels)
    return files, labels

dir = "C:/Users/s141554/Documents/Systems Biology/Computational neuroscience/cochleagrams-preprocessed"
data_coch, labels_coch = load_files(dir)
train_data_ch, test_data_ch, train_label_ch, test_label_ch = train_test_split(data_coch, labels_coch, test_size=0.2, random_state=42)



C:\Users\s141554\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\s141554\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\s141554\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\s141554\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

1.14.0


In [2]:
#Input - make sure for every number the amount of visual and auditory presentations are the same 
def match_data(data_mnist, labels_mnist, data_coch, labels_coch):
    data_ch = []
    data_m = []
    labels = []
    for x in range(10):
        nr_mnist = len(labels_mnist[labels_mnist == x])
        nr_coch = len(labels_coch[labels_coch == x])
        reps = np.ceil(nr_mnist/nr_coch)

        rep_data_ch = np.repeat(data_coch[labels_coch == x], reps, axis=0)
        rep_data_ch = rep_data_ch[:nr_mnist]
        data_ch.append(rep_data_ch)

        data_m.append(data_mnist[labels_mnist == x])
        labels.append(labels_mnist[labels_mnist == x])

    data_ch = np.concatenate(data_ch, axis=0)
    data_m = np.concatenate(data_m, axis=0)
    labels = np.concatenate(labels, axis=0)
    return data_ch, data_m, labels
#output (created with match_data)
train_data_cochleagrams, train_data_visual, train_label = match_data(train_imgs_mnist, train_labs_mnist, train_data_ch, train_label_ch)
test_data_cochleagrams, test_data_visual, test_label = match_data(test_imgs_mnist, test_labs_mnist, test_data_ch, test_label_ch)

#create random index order

index_visual=np.arange(60000)
np.random.shuffle(index_visual)

index_auditory=np.arange(60000)
np.random.shuffle(index_auditory)


index_visual_test = np.arange(10000)
np.random.shuffle(index_visual_test)
index_auditory_test = np.arange(10000)
np.random.shuffle(index_auditory_test)
#randomize the order of the cochleagrams and mnist while keeping them coupled to the right label

train_cochleagrams = train_data_cochleagrams[index_auditory]
train_mnist= train_data_visual[index_visual]
train_labels_cochleagrams = train_label[index_auditory]
train_labels_mnist = train_label[index_visual]


test_cochleagrams = test_data_cochleagrams[index_auditory_test]
test_mnist = test_data_visual[index_visual_test]
test_labels_cochleagrams = test_label[index_auditory_test]
test_labels_mnist = test_label[index_visual_test]

In [4]:
### Prepare data for the task classifiers 
#Break training data into blocks
vis1 = train_mnist[:30000]
vis2 = train_mnist[30000:]
vis1_lab = train_labels_mnist[:30000] 
vis2_lab = train_labels_mnist[30000:]

aud1 = train_cochleagrams[:30000]
aud2 = train_cochleagrams[30000:]
aud1_lab = train_labels_cochleagrams[:30000]
aud2_lab = train_labels_cochleagrams[30000:]

zeros = np.zeros((30000,), dtype=int)
empty_mnist = np.zeros((28,28),dtype=int)
zeros_mnist = np.repeat([empty_mnist], 30000, axis=0)
empty_coch = np.zeros((15,53),dtype=int)
zeros_coch = np.repeat([empty_coch], 30000, axis =0)

#create 4 input streams
## vis1  vis2  0    0 
## vis2   0    0    aud1
## 0     vis1 aud2  0 
## 0      0   aud1  aud2
in_vis_1 = np.concatenate((vis1, vis2, zeros_mnist, zeros_mnist))
labels_in_vis_1 = np.concatenate((vis1_lab, vis2_lab, zeros, zeros))

in_vis_2 = np.concatenate((vis2, zeros_mnist, vis1, zeros_mnist))
labels_in_vis_2 = np.concatenate((vis2_lab, zeros, vis1_lab, zeros))

in_aud_1 = np.concatenate((zeros_coch, zeros_coch, aud2, aud1))
labels_in_aud_1 = np.concatenate((zeros, zeros, aud2_lab, aud1_lab))

in_aud_2 = np.concatenate((zeros_coch, aud1, zeros_coch, aud2))
labels_in_aud_2 = np.concatenate((zeros, aud1_lab, zeros, aud2_lab))

training_input = [in_vis_1, in_vis_2, in_aud_1, ]

#Repeat with test data
vis1_test = test_mnist[:5000]
vis2_test = test_mnist[5000:]
vis1_lab_test = test_labels_mnist[:5000]
vis2_lab_test = test_labels_mnist[5000:]

aud1_test = test_cochleagrams[:5000]
aud2_test = test_cochleagrams[5000:]
aud1_lab_test = test_labels_cochleagrams[:5000]
aud2_lab_test = test_labels_cochleagrams[5000:]

zeros_test = np.zeros((5000,), dtype=int)
zeros_mnist_test = np.repeat([empty_mnist], 5000, axis=0)
zeros_coch_test = np.repeat([empty_coch], 5000, axis =0)

#create 4 input streams
## vis1  vis2  0    0 
## vis2   0    0    aud1
## 0     vis1 aud2  0 
## 0      0   aud1  aud2

in_vis_1_test = np.concatenate((vis1_test, vis2_test, zeros_mnist_test, zeros_mnist_test))
labels_in_vis_1_test = np.concatenate((vis1_lab_test, vis2_lab_test, zeros_test, zeros_test))

in_vis_2_test = np.concatenate((vis2_test, zeros_mnist_test, vis1_test, zeros_mnist_test))
labels_in_vis_2_test = np.concatenate((vis2_lab_test, zeros_test, vis1_lab_test, zeros_test))

in_aud_1_test = np.concatenate((zeros_coch_test, zeros_coch_test, aud2_test, aud1_test))
labels_in_aud_1_test = np.concatenate((zeros_test, zeros_test, aud2_lab_test, aud1_lab_test))

in_aud_2_test = np.concatenate((zeros_coch_test, aud1_test, zeros_coch_test, aud2_test))
labels_in_aud_2_test = np.concatenate((zeros_test, aud1_lab_test, zeros_test, aud2_lab_test))




In [6]:
#OLD VERSION 
#Input Visual 1
input_visual_1 = keras.layers.Input(shape=(28,28))
flatten_vis_1 = keras.layers.Flatten(input_shape=(28,28))(input_visual_1)
l1_vis_1 = keras.layers.Dense(32, activation='relu')(flatten_vis_1)

#Input Visual 2
input_visual_2 = keras.layers.Input(shape=(28,28))
flatten_vis_2 = keras.layers.Flatten(input_shape=(28,28))(input_visual_2)
l1_vis_2 = keras.layers.Dense(32, activation='relu')(flatten_vis_2)

#Input auditory 1
input_auditory_1 = keras.layers.Input(shape=(15,53))
flatten_aud_1 = keras.layers.Flatten(input_shape=(15,53))(input_auditory_1)
l1_aud_1 = keras.layers.Dense(32, activation='relu')(flatten_aud_1)

#Input auditory 2
input_auditory_2 = keras.layers.Input(shape=(15,53))
flatten_aud_2 = keras.layers.Flatten(input_shape=(15,53))(input_auditory_2)
l1_aud_2 = keras.layers.Dense(32, activation='relu')(flatten_aud_2)

#Concatenated
concatenated = keras.layers.Concatenate()([l1_vis_1, l1_vis_2, l1_aud_1, l1_aud_2])
l2 = keras.layers.Dense(64, activation = 'relu')(concatenated)
l3 = keras.layers.Dense(32, activation = 'relu')(l2)
out= keras.layers.Dense(19)(l3)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
from tensorflow.python.keras import Model
class Sequentialmodel(Model):
    def __init__(self):
        super(Sequentialmodel, self).__init__()
        #Input Visual  sequential
        self.input_mnist1 = tf.keras.Sequential([
            keras.layers.Flatten(input_shape=(28,28)),
            keras.layers.Dense(32, activation='relu')
        ])
        
        #Input Visual  sequential
        self.input_mnist2 = tf.keras.Sequential([
            keras.layers.Flatten(input_shape=(28,28)),
            keras.layers.Dense(32, activation='relu')
        ])


        #Input auditory sequential
        self.input_coch1 = tf.keras.Sequential([
            keras.layers.Flatten(input_shape=(15,53)),
            keras.layers.Dense(32, activation='relu')
        ])
        
         #Input auditory sequential
        self.input_coch2 = tf.keras.Sequential([
            keras.layers.Flatten(input_shape=(15,53)),
            keras.layers.Dense(32, activation='relu')
        ])

        #Concatenated sequential 
        self.concatenated = tf.keras.Sequential([
        keras.layers.Concatenate(axis=-1),
        keras.layers.Dense(64, activation = 'relu'),
        keras.layers.Dense(32, activation = 'relu')
        ])
        
        #output sequential
        self.out= tf.keras.Sequential([keras.layers.Dense(19)])
        

    
    def encode(self, im1, im2, ch1, ch2):
        encoded_im1 = self.input_mnist1(im1) 
        encoded_im2 = self.input_mnist2(im2)
        encoded_ch1 = self.input_coch1(ch1)
        encoded_ch2 = self.input_coch2(ch2)
        combined_encoding = self.concatenated([encoded_im1, encoded_im2, encoded_ch1, encoded_ch2])
        return combined_encoding
    
    def output(self, encoded):
        out = self.out(encoded)
        return out
    
    def call(self, input):
        im1, im2, ch1, ch2 = input
        encoding = self.encode(im1, im2, ch1, ch2)
        out = self.output(encoding)
        return out
        
        

In [6]:
#Addition ....
train_labs_addition_visual_cochleagram = labels_in_vis_1 + labels_in_vis_2 + labels_in_aud_1 + labels_in_aud_2
print(train_labs_addition_visual_cochleagram)
print(max(train_labs_addition_visual_cochleagram))
print(min(train_labs_addition_visual_cochleagram))

test_labs_addition_visual_cochleagram = labels_in_vis_1_test + labels_in_vis_2_test + labels_in_aud_1_test + labels_in_aud_2_test
print(test_labs_addition_visual_cochleagram)
print(max(test_labs_addition_visual_cochleagram))
print(min(test_labs_addition_visual_cochleagram))

[16  3  6 ...  9 13 10]
18
0
[ 6  9  6 ... 10  7  5]
18
0


In [7]:
#Addition model sequential
model_add = Sequentialmodel()
model_add.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
model_add.fit([in_vis_1, in_vis_2, in_aud_1, in_aud_2], train_labs_addition_visual_cochleagram, epochs = 5, shuffle=True)

#Evaluate model addition
test_loss, test_acc = model_add.evaluate([in_vis_1_test, in_vis_2_test, in_aud_1_test, in_aud_2_test],  test_labs_addition_visual_cochleagram, verbose=2)
print('test acc:', test_acc)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/5
120000/120000 [==============================] - 10s 79us/sample - loss: 1.9700 - acc: 0.2985
Epoch 2/5
120000/120000 [==============================] - 9s 73us/sample - loss: 0.9743 - acc: 0.6720
Epoch 3/5
120000/120000 [==============================] - 9s 72us/sample - loss: 0.5967 - acc: 0.8172
Epoch 4/5
120000/120000 [==============================] - 8s 71us/sample - loss: 0.4105 - acc: 0.8857
Epoch 5/5
120000/120000 [==============================] - 9s 72us/sample - loss: 0.3065 - acc: 0.9184
20000/20000 - 1s - loss: 1.7466 - acc: 0.6335
test acc: 0.6335


In [6]:
#OLD VERSION 
#Addition model
model_addition = keras.models.Model(inputs=[input_visual_1, input_visual_2, input_auditory_1, input_auditory_2], outputs = out)
model_addition.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
params_model_addition = model_addition.fit([in_vis_1, in_vis_2, in_aud_1, in_aud_2], train_labs_addition_visual_cochleagram, epochs = 5, shuffle=True)

#Evaluate model addition
test_loss, test_acc = model_addition.evaluate([in_vis_1_test, in_vis_2_test, in_aud_1_test, in_aud_2_test],  test_labs_addition_visual_cochleagram, verbose=2)
print('test acc:', test_acc)


Epoch 1/5
120000/120000 [==============================] - 26s 219us/sample - loss: 1.9733 - acc: 0.3046
Epoch 2/5
120000/120000 [==============================] - 20s 167us/sample - loss: 0.9977 - acc: 0.6648
Epoch 3/5
120000/120000 [==============================] - 18s 153us/sample - loss: 0.6355 - acc: 0.7985
Epoch 4/5
120000/120000 [==============================] - 16s 136us/sample - loss: 0.4561 - acc: 0.8665
Epoch 5/5
120000/120000 [==============================] - 14s 120us/sample - loss: 0.3467 - acc: 0.9042
20000/20000 - 1s - loss: 1.6716 - acc: 0.6263
test acc: 0.6263


KeyError: 'accuracy'

In [12]:
class Classifier(Model):
    def __init__(self, model):
        super(Classifier, self).__init__()
        # because we have a binary classifier, sigmoid is more suitable
        activation = 'sigmoid'
        # self.model: keras.Model = tf.keras.models.load_model(dir, custom_objects={"seperate_loss": seperate_loss})
        self.model = model
        for layer in self.model.layers:
            layer.trainable = False

        self.classifier = tf.keras.Sequential([
            keras.layers.Dense(1, activation=activation)  # latent space
        ])

    def call(self, input):
        im1, im2, ch1, ch2 = input
        latent_space = self.model.encode(im1, im2, ch1, ch2)
        classified = self.classifier(latent_space)
        return classified


def make_dataset(data_im, data_ch):
    empty_mnist = np.zeros((28,28),dtype=int)
    empty_coch = np.zeros((15,53),dtype=int)
    data_im_zeros = np.repeat([empty_mnist], int(len(data_im)/2), axis=0)
    data_ch_zeros = np.repeat([empty_coch], int(len(data_im)/2), axis=0)
    
    data_im1 = data_im[:int(len(data_im)/2)]
    data_im2 = data_im[int(len(data_im)/2):]
    data_ch1 = data_ch[:int(len(data_im)/2)]
    data_ch2 = data_ch[int(len(data_im)/2):]
    

    data_1 = np.concatenate([data_im1, data_im2, data_im_zeros, data_im_zeros], axis=0)
    data_2 = np.concatenate([data_im2, data_im1, data_im_zeros, data_im_zeros], axis=0)
    data_3 = np.concatenate([data_ch_zeros, data_ch_zeros, data_ch1, data_ch2], axis=0)
    data_4 = np.concatenate([data_ch_zeros, data_ch_zeros, data_ch2, data_ch1], axis=0)

    #if input is an image, output 1, if input is a cochleagram output 0
    im_lab = np.ones(len(data_im))
    ch_lab = np.zeros(len(data_ch))
    labels = np.concatenate([im_lab, ch_lab])

    
    return data_1, data_2, data_3, data_4, labels



In [13]:
data_ls_1, data_ls_2, data_ls_3, data_ls_4, train_labels_latent_space = make_dataset(train_mnist, train_cochleagrams)
data_testls_1, data_testls_2, data_testls_3, data_testls_4, test_labels_latent_space = make_dataset(test_mnist, test_cochleagrams)
print(train_labels_latent_space)

classifier = Classifier(model_add)
classifier.compile(optimizer='adam',
                       loss=keras.losses.BinaryCrossentropy(from_logits=True),
                       # since input are values, no probabilities
                       metrics=['accuracy'])

classifier.fit([data_ls_1, data_ls_2, data_ls_3, data_ls_4], train_labels_latent_space, epochs=5, shuffle=True)
test_loss, test_acc = classifier.evaluate([data_testls_1, data_testls_2, data_testls_3, data_testls_4], test_labels_latent_space, verbose=2)

[1. 1. 1. ... 0. 0. 0.]


TypeError: in converted code:

    <ipython-input-12-32885d1d9cb1>:17 call
        latent_space = self.model.encode(self, im1, im2, ch1, ch2)

    TypeError: encode() takes 5 positional arguments but 6 were given
